In [ ]:
from pymongo import MongoClient
client = MongoClient(host="54.69.198.239",port=27017)
db = client.UFO
col = db.Summaries
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import word2vec
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)
from sklearn.metrics.pairwise import linear_kernel

from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import time
import pandas as pd
from gensim.models import word2vec

In [ ]:
df_geocode = pd.read_csv("52150_locations_USA_census_nonull.csv")
del df_geocode['Unnamed: 0']

In [ ]:
#create list of bag_of_words from each summary
num = 10000
pipeline = [{"$limit":num}]
test_texts = []
t = time.time()
for i in col.aggregate(pipeline):
    test_texts.append(i)
print time.time() - t

In [ ]:
#stem each word and return to list of bag_of_words separated by spaces
#each element of list is a summary

t = time.time()
bag_of_words = []
for text in test_texts:
    stemmed_words = []
    words = text['bag_of_words'][0].split()
    for w in words:
        stemmed_words.append(porter_stemmer.stem(w))
    bag_of_words.append(" ".join(stemmed_words))
print time.time() - t

In [ ]:
#Count vectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = 'english',   \
                             max_features = 3000) 

X_ct = vectorizer.fit_transform(bag_of_words)
X_ct = X_ct.toarray()

print X_ct.shape

vocab = vectorizer.get_feature_names()
dist = np.sum(X_ct, axis=0) #use dist and vocab to print word count

In [ ]:
t = time.time()
vectorizer_tf = TfidfVectorizer(stop_words = "english")
X = vectorizer_tf.fit_transform(bag_of_words)

#pca = PCA()
#X_pca = pca.fit_transform(X.toarray())

#shows pairwise similarity between each text
#print (X * X.T).A

#inertia = []
#ks = range(1,num+1) #choose cluser number
#for k in ks:
#    model = KMeans(n_clusters=k, init='k-means++', max_iter=500, n_init=1, random_state=1)
#    m = model.fit(X)
#    inertia.append(m.inertia_)
    #print "Intertia: {0}".format(m.inertia_)
print time.time() - t

In [ ]:
cosine_similarities = linear_kernel(X[1:2], X).flatten()

In [ ]:
related_docs_indices = cosine_similarities.argsort()[:-5:-1]

In [ ]:
cos_sim = (X * X.T).A

In [ ]:
inds = cos_sim.argmin(1)
for i,ind in enumerate(inds):
    print bag_of_words[i]+"\n"+bag_of_words[ind]
    print

In [ ]:
model = KMeans(n_clusters=100, init='k-means++', max_iter=500, n_init=1, random_state=1)
m = model.fit(X)
centers = m.cluster_centers_
centers.shape

In [ ]:
d = m.transform(X)[:, 11] #the distance to the j'th centroid for each point in an array X
ind = np.argsort(d)[::-1][:1]
print bag_of_words[ind]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(X)

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

#X_scaled = StandardScaler().fit_transform(X.toarray())

db = DBSCAN(eps=.5, min_samples=2).fit(X.toarray())
#core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
#core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

###Word2Vec

In [ ]:
# Set values for various parameters
#num_features =     # Word vector dimensionality                      
min_word_count = 30   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [ ]:
model = word2vec.Word2Vec(sents, workers=num_workers, \
            min_count = min_word_count, \
            window = context, sample = downsampling)

In [ ]:
model.init_sims(replace=True)

In [ ]:
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5 #332
print num_clusters

In [ ]:
t = time.time()
kmeans_clustering = KMeans(n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )
print time.time() - t

In [ ]:
word_centroid_map = dict(zip( model.index2word, idx ))

In [ ]:
for cluster in xrange(0,15):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words

In [ ]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [ ]:
model.similarity("mother","father")